# Setup

In [244]:
# # Init pyspark
# from pyspark import SparkContext
# sc = SparkContext.getOrCreate()
# # Init sparksql -- Only used to format the output nicely!
# from pyspark.sql import SQLContext
# sqlContext = SQLContext(sc)

# rows = sc.textFile("/air_transit_2007.csv")
# #rows = sc.textFile("/Users/abel/Downloads/spare_time/working/a13544_600_spark/air_transit_2007.csv")

# data = rows.map(lambda line: line.split(","))
# # data.cache()

# import findspark
# findspark.init()
# A simple demo for working with SparkSQL and Tweets
from pyspark import SparkContext, SparkConf
from pyspark.sql import HiveContext, Row
from pyspark.sql.types import IntegerType
import json
import sys


inputFile = 'air_transit_2007.csv'
#inputFile = '/Users/abel/Downloads/spare_time/working/a13544_600_spark/air_transit_2007.csv'

conf = SparkConf().setAppName("SparkSQLAirTransit")
sc = SparkContext.getOrCreate(conf=conf)
hiveCtx = HiveContext(sc)
print("Loading tweets from " + inputFile)


Loading tweets from air_transit_2007.csv


In [245]:
input = hiveCtx.read.option("header",True).csv(inputFile,inferSchema =True)
input.printSchema()



root
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- DepTime: string (nullable = true)
 |-- CRSDepTime: integer (nullable = true)
 |-- ArrTime: string (nullable = true)
 |-- CRSArrTime: integer (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- FlightNum: integer (nullable = true)
 |-- TailNum: string (nullable = true)
 |-- ActualElapsedTime: string (nullable = true)
 |-- CRSElapsedTime: integer (nullable = true)
 |-- AirTime: string (nullable = true)
 |-- ArrDelay: string (nullable = true)
 |-- DepDelay: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: integer (nullable = true)
 |-- TaxiIn: integer (nullable = true)
 |-- TaxiOut: integer (nullable = true)
 |-- Cancelled: integer (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Diverted: integer (nullable = true)
 |-- 

In [246]:
# Sample Query

In [247]:

input.registerTempTable("air_transit")

myair_transits = hiveCtx.sql("SELECT * FROM air_transit  LIMIT 3")
print('myair_transits:' )
for item in myair_transits.collect():
    print(item, '\n')

myair_transits:
Row(Year=2007, Month=1, DayofMonth=1, DayOfWeek=1, DepTime='1232', CRSDepTime=1225, ArrTime='1341', CRSArrTime=1340, UniqueCarrier='WN', FlightNum=2891, TailNum='N351', ActualElapsedTime='69', CRSElapsedTime=75, AirTime='54', ArrDelay='1', DepDelay='7', Origin='SMF', Dest='ONT', Distance=389, TaxiIn=4, TaxiOut=11, Cancelled=0, CancellationCode=None, Diverted=0, CarrierDelay=0, WeatherDelay=0, NASDelay=0, SecurityDelay=0, LateAircraftDelay=0) 

Row(Year=2007, Month=1, DayofMonth=1, DayOfWeek=1, DepTime='1918', CRSDepTime=1905, ArrTime='2043', CRSArrTime=2035, UniqueCarrier='WN', FlightNum=462, TailNum='N370', ActualElapsedTime='85', CRSElapsedTime=90, AirTime='74', ArrDelay='8', DepDelay='13', Origin='SMF', Dest='PDX', Distance=479, TaxiIn=5, TaxiOut=6, Cancelled=0, CancellationCode=None, Diverted=0, CarrierDelay=0, WeatherDelay=0, NASDelay=0, SecurityDelay=0, LateAircraftDelay=0) 

Row(Year=2007, Month=1, DayofMonth=1, DayOfWeek=1, DepTime='2206', CRSDepTime=2130, ArrTi

## Q1
Compute the total number of records.

In [248]:
# Response...
mycount = hiveCtx.sql("SELECT count(*) as mycount FROM air_transit")
print('mycount=', mycount.collect()[0]['mycount'] )

mycount= 756


In [249]:
## Q2
#Find total number of operated flights per month, sorted by the month..


mymonth = hiveCtx.sql("select Month, count(*) as flight_number from air_transit group by Month order by Month LIMIT 100")
print('total number of operated flights per month:')  
for item in mymonth.collect():
    print(item['Month'], 'month', item['flight_number'])

total number of operated flights per month:
1 month 100
2 month 249
12 month 407


In [250]:
## Q3
# Find the plane with the highest number of flights. Each plane has a unique TailNum
highest_number_filghts = hiveCtx.sql("SELECT TailNum, SUM (FlightNum) FROM air_transit GROUP BY TailNum LIMIT 100")
print('highest_number_filghts: ', highest_number_filghts.collect()[0]['TailNum'] )

highest_number_filghts:  N6700


In [251]:
## Q4
# Compute the total flight time of each airplane, sorted by flight time in descending order.
total_flight_times = hiveCtx.sql("SELECT TailNum, SUM (AirTime) as total_fight_time FROM air_transit GROUP BY TailNum order by total_fight_time desc LIMIT 100")
print('-'*10, 'We make ', '-'*10)
print('total flight time of each airplane: ')
for item in  total_flight_times.collect():
    print(item['TailNum'], ' total fight time is', item['total_fight_time'])


---------- We make  ----------
total flight time of each airplane: 
N3767  total fight time is 907.0
N385DN  total fight time is 804.0
N6707A  total fight time is 743.0
N3731T  total fight time is 696.0
N904DE  total fight time is 696.0
N377DA  total fight time is 685.0
N391DA  total fight time is 681.0
N830MH  total fight time is 661.0
N925DL  total fight time is 640.0
N948DL  total fight time is 627.0
N946DL  total fight time is 595.0
N645DL  total fight time is 577.0
N912DL  total fight time is 563.0
N394DA  total fight time is 553.0
N6709  total fight time is 548.0
N718SW  total fight time is 544.0
N135DL  total fight time is 521.0
N980DL  total fight time is 485.0
N6702  total fight time is 482.0
N995DL  total fight time is 478.0
N911DL  total fight time is 477.0
N3766  total fight time is 473.0
N372DA  total fight time is 449.0
N655DL  total fight time is 443.0
N988DL  total fight time is 430.0
N931DL  total fight time is 423.0
N828MH  total fight time is 414.0
N654DL  total figh

In [252]:
# q5 Find the busiest airport (in terms of number of departures + arrivals of all operated flights) for each month.
from pyspark.sql.functions import col
import operator
df = input

for month in range(1, 12+1):
    print(month, '-'*10)
    df1=df.filter(col('Month').isin([month])).groupBy('Origin').count()
    df2=df.filter(col('Month').isin([month])).groupBy('Dest').count()
    #print(type(df1), type(df2))
    k_v = {}
    for orgin in df1.collect():
#         print(orgin['Origin'],orgin['count'])
        k_v[orgin['Origin']] = orgin['count']
    for dest in df2.collect():
        if k_v.get(dest['Dest']) :
#             print(dest)
#             print('before',k_v[dest['Dest']])
            k_v[dest['Dest']] += dest['count']
#             print('after',k_v[dest['Dest']])

    print(k_v)
    if k_v is None:
        month_max = max(k_v.items(), key=operator.itemgetter(1))[0]
        print('month_max:', month_max)



1 ----------
{'SNA': 43, 'SMF': 43, 'STL': 28}
2 ----------
{'LAS': 161, 'LAX': 102}
3 ----------
{}
4 ----------
{}
5 ----------
{}
6 ----------
{}
7 ----------
{}
8 ----------
{}
9 ----------
{}
10 ----------
{}
11 ----------
{}
12 ----------
{'MSY': 10, 'GEG': 2, 'SNA': 8, 'DCA': 32, 'ORF': 2, 'SAV': 4, 'CMH': 1, 'PNS': 3, 'IAH': 3, 'HNL': 3, 'CVG': 32, 'LGA': 88, 'AUS': 1, 'SJU': 1, 'SRQ': 3, 'CHS': 6, 'RSW': 7, 'BOS': 57, 'EWR': 4, 'LAS': 10, 'DEN': 5, 'IAD': 1, 'BOI': 3, 'SEA': 5, 'MCI': 3, 'CLT': 8, 'PBI': 13, 'ABQ': 3, 'SDF': 2, 'BDL': 4, 'PDX': 1, 'MIA': 5, 'TPA': 27, 'BWI': 4, 'SMF': 2, 'PHX': 5, 'STL': 2, 'DFW': 5, 'GSP': 2, 'SFO': 11, 'MEM': 2, 'BHM': 5, 'ATL': 197, 'FLL': 22, 'RIC': 3, 'VPS': 2, 'LIT': 1, 'ORD': 5, 'RDU': 6, 'MKE': 2, 'HSV': 2, 'PIT': 5, 'IND': 3, 'TYS': 2, 'ONT': 3, 'JAX': 6, 'LAX': 24, 'MCO': 39, 'ROC': 2, 'SAN': 4, 'JFK': 19, 'DAB': 1, 'PHL': 7, 'SAT': 5, 'SLC': 50}


In [253]:
## Q6
# 6nd the airline with highest average delay of each type in March 2007. 
#Note: do not write separate code for each error type. 
# You should compute a single RDD where each row contains the delay type, 
# the airline that is worst regarding that delay type, and its average delay of that type in minutes.

import pyspark.sql.functions as F 
from pyspark.sql.functions import count, avg
from pyspark.sql.functions import  max as max_

# d_columns =  ['CarrierDelay','WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay']
print('1 all of group:')
df1 = df.groupBy("FlightNum") \
    .avg('CarrierDelay','WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay')
df1.show()

r = df1.groupBy("FlightNum").agg(F.sum(df1[1]+df1[2] + df1[3] + df1[4]).alias('result')).orderBy('result',ascending=False).head(1)
print('2 the filghtnum is:', r[0].FlightNum)

print('3 the max row:')
df1.filter(col('FlightNum').isin([r[0].FlightNum])).show()

1 all of group:
+---------+-----------------+-----------------+-------------+------------------+----------------------+
|FlightNum|avg(CarrierDelay)|avg(WeatherDelay)|avg(NASDelay)|avg(SecurityDelay)|avg(LateAircraftDelay)|
+---------+-----------------+-----------------+-------------+------------------+----------------------+
|     1959|              0.0|              0.0|          0.0|               0.0|                   0.0|
|     2659|              0.0|              0.0|          0.0|               0.0|                   0.0|
|     1238|              0.0|              0.0|          0.0|               0.0|                   0.0|
|     1591|              0.0|              0.0|          0.0|               0.0|                   0.0|
|     1645|              0.0|              0.0|          0.0|               0.0|                   0.0|
|     1460|              0.0|              0.0|          0.0|               0.0|                   0.0|
|     1522|              0.0|              0.0| 

In [255]:
## q7
# Compute median, mean, and mode of columns 12-16, 19-21 and 25-29 
# for the flights in the third week of 2007. Exclude the non-numeric values.

from pyspark.sql import functions as F

print('filter out week 3')
mydf = df.filter(col('DayOfWeek').isin([3])) 


df1 = mydf.select(mydf.columns[12:16])
df1.show()
# res  = df1.select(*[F.percentile_approx(c).alias(c) for c in df1.columns])
# res.show()
# for name in ('CRSElapsedTime', 'AirTime', 'ArrDelay', 'DepDelay'):

quantiles = df1.approxQuantile("CRSElapsedTime", [0.25, 0.5, 0.75], 0)
print('median of [12:16]:')
print(quantiles)

df2 = mydf.select(mydf.columns[19:21])
# df2.show()
res2  = df2.select(*[F.mean(c).alias(c) for c in df2.columns])
print('mean of [19:21] :')
res2.show()

df3 = mydf.select(mydf.columns[25:29])
# df3.show()
print('mode of [25:29] :')
# [df3.groupby(i).count().orderBy("count", ascending=False).first()[0] for i in df3.columns]
for i in df3.columns:
    t = df3.groupby(i).count().orderBy("count", ascending=False).first()
    if t is not None:
        print(t[0])

filter out week 3
+--------------+-------+--------+--------+
|CRSElapsedTime|AirTime|ArrDelay|DepDelay|
+--------------+-------+--------+--------+
+--------------+-------+--------+--------+

median of [12:16]:
[]
mean of [19:21] :
+------+-------+
|TaxiIn|TaxiOut|
+------+-------+
|  null|   null|
+------+-------+

mode of [25:29] :


In [274]:
## Q8
#Assume that a passenger wants to travel from Philadelphia International Airport (airport code: PHL) 
# to Los Angeles International Airport (airport code: LAX), and then go back to Philadelphia (PHL). 
# He departs PHL not earlier than 5:59 am (scheduled time), stays at least 3:01 hours in Los Angeles and then arrive at PHL not later than 11pm. Based on the "scheduled" times, find which carrier has the highest number of flights with these constraints. 
# Limit your analysis to February 2007 and use scheduled times.


from pyspark.sql.functions import col
import pyspark.sql.functions as F
df1 = df.filter(col('Month').isin([2])) 
df1 = df1.filter(col('Origin').isin(['PHL', 'LAX'])).filter(col('Dest').isin(['PHL', 'LAX']))
df1.show()

# df2 = df1.filter(col('DepTime').isin([2])) 
print('stay 3:0 hour, it means before we shoulf flight here at least before back-fight start time - 3hours')
dates = ("1970-01-01 5:59:00", "1970-01-01 20:00:00", "1970-01-01 23:00:00")
# date_from, date_to = [to_date(lit(s)).cast(TimestampType()) for s in dates]


df2 = df1.withColumn('DepTime_Date', F.to_timestamp('DepTime', 'hhmm'))
df3 = df2.withColumn('ArrTime_Date', F.to_timestamp('ArrTime', 'hhmm'))
# df2.show()

df4 = df3.where( ((col('DepTime_Date') > dates[0]) & (col('ArrTime_Date') < dates[1])) |
           (col('ArrTime_Date') < dates[2])
         ).show(truncate=False)

if df4 is not None:
    df5 = df4.groupBy('UniqueCarrier').count()
    k_v = {}
    for orgin in df5.collect():

        k_v[orgin['UniqueCarrier']] = orgin['count']
    print(k_v)
    if k_v is None:
        carriar_max = max(k_v.items(), key=operator.itemgetter(1))[0]
        print('max UniqueCarrier :', carriar_max)

+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|TaxiOut|Cancelled|CancellationCode|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|
+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|2007|    2|         1|        4|   1337|      1335|   2114|      2135|           WN|      110

In [295]:
## Q9
#Generate the `departure flights` board of the Los Angeles Airport at 12 Jan 2007 at 13:00. 
# The board should contain flights with actual departure times between 12:00 and 14:00, 
# sorted by scheduled departure time. The resulting table should at least contain scheduled departure time, 
# actual departure time (if departed), 
# airline code, and destination
df1 = df.filter(col('Month').isin([1])).filter(col('DayofMonth').isin([1]))

dates = ("1970-01-01 12:00:00", "1970-01-01 14:00:00")
dates = ("1970-01-01 00:01:00", "1970-01-01 23:59:00")
# date_from, date_to = [to_date(lit(s)).cast(TimestampType()) for s in dates]

df1 = df1.withColumn(
    "CRSDepTime",
    F.when(F.col("CRSDepTime").isNull(), '0')
    .otherwise(F.col("CRSDepTime").cast("string"))
)
df2 = df1.withColumn('DepTime_Date', F.to_timestamp('CRSDepTime', 'hhmm'))

# df2.show()
# print('@'*20)

df4 = df2.where( 
    ((col('DepTime_Date') > dates[0]) & (col('DepTime_Date') < dates[1]))
         )

print('-'*10, 'departure flights', '-'*10)
for item in df4.rdd.collect():
#     print(item)
    print(item['CRSArrTime'],item['DepTime'], item['FlightNum'], item['Dest'])

---------- departure flights ----------
1340 1232 2891 ONT
1410 1251 933 SAN
1205 1042 1554 SAN
1225 1045 1502 SEA
1300 1140 1596 SNA
1300 1233 851 LAS
1155 1035 1184 OAK
1315 1101 324 PHX
1450 1245 1684 PHX
1250 1136 2037 SJC
1220 1052 1977 SMF
1340 1215 2588 SMF
1225 959 1477 CLE
1225 1058 2108 DAL
1115 1014 1361 LIT
